In [1]:
import argparse
# from dataclasses import dataclass
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate


In [2]:
CHROMA_PATH = "chroma"

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""
CHAT_MODEL = ChatOllama(model='llama3:8b')
EMBEDDING_MODEL = OllamaEmbeddings(model="mxbai-embed-large")
DB = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDING_MODEL, collection_metadata={"hnsw:space": "cosine"}, collection_name='test')

In [3]:
def get_input():
    return input("Enter your response: ")

def search_db(query_text):
    db_results = DB.similarity_search_with_relevance_scores(query_text, k=1)
    # if len(db_results) == 0 or db_results[0][1] < 0.7:
    #     print(f"Unable to find matching results.")
    #     return
    return db_results

def generate_prompt(db_results, query_text):
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in db_results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    return prompt

def generate_response(prompt):
    return CHAT_MODEL.predict(prompt)

def format_response(response_text, db_results):
    sources = [doc.metadata.get("source", None) for doc, _score in db_results]
    return f"Response: {response_text}\nSources: {sources}"

In [4]:
entry = get_input()
db_results = search_db(entry)
if db_results:
    prompt = generate_prompt(db_results, entry)
    response = generate_response(prompt)
    print(format_response(response, db_results))

/workspaces/MigraChat/.venv/lib/python3.11/site-packages/langchain_core/vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content="(b) The Secretary shall convene and preside at meetings of the Task Force, direct its work, and as appropriate, establish and direct subgroups of the Task Force that shall consist exclusively of Task Force members. The Secretary shall designate an official of the Department to serve as the Executive Secretary of the Task Force, and the Executive Secretary shall head the staff assigned to the Task Force.\nSec. 3. Functions. Consistent with applicable law, the Task Force shall:\n(a) provide direction to executive departments and agencies (agencies) concerning the integration into American society of America's legal immigrants, particularly through instruction in English, civics, and history;\n(b) promote public-private partnerships that will encourage businesses to offer English and civics education to workers;", 

Response: According to the provided context, the powers and duties of the Secretary are:

1. Convene and preside at meetings of the Task Force.
2. Direct the work of the Task Force.
3. Establish and direct subgroups of the Task Force, consisting exclusively of Task Force members.
4. Designate an official of the Department to serve as the Executive Secretary of the Task Force.
5. Head the staff assigned to the Task Force.

These powers and duties are related to overseeing the work of the Task Force and ensuring its effective operation.
Sources: ['test_pages/156895302173890146.md']
